In [1]:
import numpy as np
from scipy.io import loadmat
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from scipy.stats import pearsonr
from scipy import spatial
from sklearn.metrics import auc, r2_score, roc_auc_score

c:\users\test\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\test\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\users\test\anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
def pairwise_accuracy(actual, predicted):
    true = 0
    total = 0
    for i in range(0,len(actual)):
#         print(i)
        for j in range(i+1, len(actual)):
            total += 1

            s1 = actual[i]
            s2 = actual[j]
            b1 = predicted[i]
            b2 = predicted[j]

            result1 = spatial.distance.cosine(s1, b1)
            result2 = spatial.distance.cosine(s2, b2)
            result3 = spatial.distance.cosine(s1, b2)
            result4 = spatial.distance.cosine(s2, b1)

            if(result1 + result2 < result3 + result4):
                true += 1

    return(true/total)

In [3]:
def pearcorr(actual, predicted):
    corr = []
    for i in range(0, len(actual)):
        corr.append(np.corrcoef(actual[i],predicted[i])[0][1])
    #np.save('correlations/'+str(i)+'.npy', corr)
    return np.mean(corr)

In [4]:
kf = KFold(n_splits=10, shuffle=True)

In [5]:
def train(vectors, voxels):
    
    dataset_X = np.array(voxels.copy())
    
    dataset_Y = np.array(vectors.copy())

#     accuracies = []
#     accuracies01 = []
    
    actual = []
    predicted = []
    pairwise_2v2 = []
    final_corr = []
    rdm_acc = []
    auc_acc = []
    r2_acc = []
    r2acc = []
    #rdm_feat = []

    cnt = 0
    for train_index, test_index in kf.split(dataset_X):

        X_train, X_test = dataset_X[train_index], dataset_X[test_index]
        y_train, y_test = dataset_Y[train_index], dataset_Y[test_index]
           
        model = Ridge(alpha=1.0)
        model.fit(X_train,y_train)
        
        

        y_pred = model.predict(X_test)
#         acc = pairwise_accuracy(y_test,y_pred)
#         accuracies.append(acc)
        
#         acc01 = rank_accuracy(y_test,y_pred)
#         accuracies01.append(acc01)
        
        pairwise_2v2.append(pairwise_accuracy(y_test,y_pred))
        final_corr.append(pearcorr(y_test,y_pred))
        cnt += 1
    return np.mean(pairwise_2v2), np.mean(final_corr)

In [6]:
img_feat = np.load('pereira_paraphrase.npy')

In [9]:
def generate_indices(data):
    Taskindices = []
    for j in data['meta'][0][0][11][0][5]:
        for k in j[0]:
            #print(k)
            Taskindices.append(int(k))
    #print(len(Taskindices))
    DMNindices = []
    for j in data['meta'][0][0][11][0][6]:
        for k in j[0]:
            #print(k)
            DMNindices.append(int(k))
    #print(len(DMNindices))
    Visualindices = []
    Visualindices_body = []
    Visualindices_face = []
    Visualindices_object = []
    Visualindices_scene = []
    for j in data['meta'][0][0][11][0][9]:
        for k in j[0]:
            #print(k)
            Visualindices_body.append(int(k))
    for j in data['meta'][0][0][11][0][10]:
        for k in j[0]:
            #print(k)
            Visualindices_face.append(int(k))
    for j in data['meta'][0][0][11][0][11]:
        for k in j[0]:
            #print(k)
            Visualindices_object.append(int(k))
    for j in data['meta'][0][0][11][0][12]:
        for k in j[0]:
            #print(k)
            Visualindices_scene.append(int(k))
    
    for j in data['meta'][0][0][11][0][13]:
        for k in j[0]:
            #print(k)
            Visualindices.append(int(k))
    #print(len(Visualindices))
    Languageindices_lh = []
    Languageindices_rh = []
    for j in data['meta'][0][0][11][0][7]:
        for k in j[0]:
            #print(k)
            Languageindices_lh.append(int(k))
    for j in data['meta'][0][0][11][0][8]:
        for k in j[0]:
            #print(k)
            Languageindices_rh.append(int(k))
        #Languageindices.append(int(j))
    #print(len(Languageindices))
    return Taskindices, DMNindices, Visualindices_body, Visualindices_face, Visualindices_object,Visualindices_scene, Visualindices, Languageindices_lh, Languageindices_rh

In [10]:
ROIS = ['language_lh', 'language_rh', 'vision_body', 'vision_face', 'vision_object',
        'vision_scene','vision', 'dmn', 
        'task']

In [11]:
def get_subject_data(subject):
    data_pic = loadmat('../Periera_Dataset/'+subject+'/data_384sentences.mat')
    data_pic1 = loadmat('../Periera_Dataset/'+subject+'/data_243sentences.mat')
    Taskindices, DMNindices, Visualindices_body, Visualindices_face, Visualindices_object,Visualindices_scene, Visualindices, Languageindices_lh, Languageindices_rh = generate_indices(data_pic)
    Taskindices1, DMNindices1, Visualindices_body1, Visualindices_face1, Visualindices_object1,Visualindices_scene1, Visualindices1, Languageindices_lh1, Languageindices_rh1 = generate_indices(data_pic1)
    roi_indices = {'language_lh':Languageindices_lh, 'language_rh':Languageindices_rh, 
        'vision_body': Visualindices_body, 'vision_face':Visualindices_face, 
        'vision_object': Visualindices_object, 'vision_scene': Visualindices_scene,
        'vision': Visualindices,  'dmn': DMNindices, 'task': Taskindices}
    
    roi_indices1 = {'language_lh':Languageindices_lh1, 'language_rh':Languageindices_rh1, 
        'vision_body': Visualindices_body1, 'vision_face':Visualindices_face1, 
        'vision_object': Visualindices_object1, 'vision_scene': Visualindices_scene1,
        'vision': Visualindices1,  'dmn': DMNindices1, 'task': Taskindices1}
    
    fmri = {}
    for roi,indices in roi_indices.items():
        a = data_pic['examples_passagesentences'][0:,np.array(indices)-1]
        b = data_pic1['examples_passagesentences'][0:,np.array(roi_indices1[roi])-1]
        #print(np.concatenate([a,b], axis=0).shape)
        fmri[roi] = np.concatenate([a,b], axis=0)
    
    
#     data = np.load('../data/features/periera/resnet_feat.npy',allow_pickle=True)
#     d = dict(enumerate(data.flatten(), 1))[1]
        
    return fmri

In [12]:
subjects = ['P01', 'M02','M04','M07', 'M15']

In [13]:
#kf = KFold(n_splits=18, shuffle=True)
kf = KFold(n_splits=5)
for roi in ROIS:
    temp1 = []
    temp2 = []
    for eachsub in subjects:
        fmri =  get_subject_data(eachsub)
        d, c = train(fmri[roi],img_feat)
        #print(np.round(d,3), np.round(c,3))
        temp1.append(d)
        temp2.append(c)
    print(np.mean(temp1), np.mean(temp2))

c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consid

0.6025248540706606 0.5700095089710672


c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consid

0.5613674961597542 0.32898261157397996


c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consid

...

0.5831153712237583 0.5456053709683739


c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consid

0.5518887455197132 0.30057756564116084


c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
c:\users\test\anaconda3\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consid

0.5525297286226319 0.2929055418191242
